In [62]:
import pandas as pd #pip install pandas
import xlwt #pip install xlwt
from openpyxl import load_workbook #pip install openpyxldd
from tqdm import tqdm
import datetime

In [76]:
rb = load_workbook('./workedjournal.xlsx')
wb = load_workbook('./DataBase.xlsx')
print(wb.sheetnames)

['Сотрудники', 'РАО', 'Отделы', 'Прочие']


In [77]:
def refresh_names():
    not_RAO_array = ['АРХПОИНТ', 'Библиотечно-информационный отдел', 'Информационный центр "Библиотека имени К.Д.Ушинского"',
                 'Сектор библиотечно-информационного обслуживания РАО', 'Сектор комплектования и обработки фонда', 'Сектор обслуживания читателей',
                 'Сектор хранения фонда']
    work_sheet_wb, work_sheet_wb_RAO, work_sheet_wb_other = wb['Сотрудники'], wb['РАО'], wb['Прочие']
    work_sheet_rb = rb['visitorReport']
    flag = False
    RAO_row, oth_row = 2, 2
    for rows in work_sheet_rb.iter_rows(values_only=True):
        if flag and rows[0][0] != '2':
            if rows[3] in not_RAO_array:
                work_sheet_wb_other.cell(row=oth_row, column=1).value = rows[0]
                work_sheet_wb_other.cell(row=oth_row, column=2).value = rows[3]
                oth_row += 1
            else:
                work_sheet_wb_RAO.cell(row=RAO_row, column=1).value = rows[0]
                work_sheet_wb_RAO.cell(row=RAO_row, column=2).value = rows[3]
                RAO_row += 1
        if rows[0] == 'Сотрудник':
            flag = True

In [78]:
# refresh_names() # Только при добавлении новых имён в БД
# wb.save('./DataBase.xlsx')

In [79]:
work_sheet_wb = wb['Сотрудники']
work_sheet_wb_RAO = wb['РАО']
work_sheet_wb_main = wb['Отделы']
work_sheet_wb_other = wb['Прочие']
work_sheet_rb = rb['visitorReport']

In [80]:
def search_data_by_name(name):
    sheet = work_sheet_rb
    n_row, n_title_row = 0, 0
    title_row = str()
    worker_dict, by_days_info = {}, {}
    in_name_flag = False

    for rows in work_sheet_rb.iter_rows(values_only=True):
        if str(rows[0])[:1] != '2':
            in_name_flag = False
        if in_name_flag:
            for i in range(len(title_row)):
                try:
                    by_days_info[title_row[i]].append(rows[i])
                except:    
                    by_days_info[title_row[i]] = [rows[i]]
        if rows[0] == 'Сотрудник':
            n_title_row = n_row
            title_row = rows
        
        if title_row and rows[0] == name:
            in_name_flag = True
            for i in range(len(title_row)):
                worker_dict[title_row[i]] = rows[i]
        n_row += 1
    return worker_dict, by_days_info

In [81]:
search_data_by_name('Баграмян Эммануил Робертович')

({'Сотрудник': 'Баграмян Эммануил Робертович',
  'Табельный номер': '',
  'Должность': 'Заместитель руководителя центра',
  'Подразделение': 'ЦЕНТР РАЗВИТИЯ ОБРАЗОВАНИЯ',
  'График работы': 'Работа с 9 до 17-45',
  'Присутствие': ' 36:38',
  'Рабочее время': ' 28:25',
  'Отсутствие': ' 13:40',
  'Работа в ночное время': '',
  'Переработка': ' 03:40',
  'Баланс отработанного времени': '-05:27',
  'Оправдательные документы с добавлением рабочего времени': '',
  'Документы без добавления времени / Сверхурочные': ''},
 {'Сотрудник': ['2022-10-04, Вт',
   '2022-10-05, Ср',
   '2022-10-06, Чт',
   '2022-10-07, Пт',
   '2022-10-08, Сб',
   '2022-10-09, Вс',
   '2022-10-10, Пн'],
  'Табельный номер': ['', '', '', '', '', '', ''],
  'Должность': ['Заместитель руководителя центра',
   'Заместитель руководителя центра',
   'Заместитель руководителя центра',
   'Заместитель руководителя центра',
   'Заместитель руководителя центра',
   'Заместитель руководителя центра',
   'Заместитель руководител

In [73]:
n_row = 0
counter = 0
days_info = []
title_dict = {
    'Сотрудник': 1,
    'График работы': 3,
    'Присутствие': 4,
    'Рабочее время': 5,
    'Отсутствие': 6,
    'Переработка': 7,
    'Баланс отработанного времени': 8
}

for rows in tqdm(work_sheet_wb_main.iter_rows(values_only=True)):
    n_row += 1

    if counter > 0:
        for key, value in days_info.items():
            try:
                work_sheet_wb_main.cell(row=n_row, column=title_dict[key]).value = value[len(value) - counter]    
            except:
                pass
    
    elif rows[1]: # Не пустая строка
        worker_dict, days_info = search_data_by_name(rows[1])
        for key, value in worker_dict.items():
            try:
                work_sheet_wb_main.cell(row=n_row, column=title_dict[key]).value = value
                counter = len(title_dict)    
            except:
                pass
    counter -= 1
    

1912it [00:34, 54.79it/s]


In [74]:
wb.save('./DataBase.xlsx')

In [11]:
worker_dict = {
    'График работы': str(),
    'Присутствие': str(),
    'Рабочее время': str(),
    'Отсутствие': str(),
    'Переработка': str(),
    'Баланс отработанного времени': str()
}

In [14]:
worker_dict['График работы'] = 'dasdas'
worker_dict['График работы']

'dasdas'

In [1]:
RAO_structure = {
    'Отдел Информатизации' : [],
    'РУКОВОДСТВО': [],
    'ОТДЕЛЕНИЕ ФИЛОСОФИИ ОБРАЗОВАНИЯ И ТЕОРЕТИЧЕСКОЙ ПЕДАГОГИКИ': [],
    'ОТДЕЛЕНИЕ ПСИХОЛОГИИ И ВОЗРАСТНОЙ ФИЗИОЛОГИИ': [],
    'ОТДЕЛЕНИЕ ОБЩЕГО СРЕДНЕГО ОБРАЗОВАНИЯ': [],
    'ОТДЕЛЕНИЕ ПРОФЕССИОНАЛЬНОГО ОБРАЗОВАНИЯ': [],
    'ОТДЕЛЕНИЕ ОБРАЗОВАНИЯ И КУЛЬТУРЫ': [],
    'ОТДЕЛЕНИЕ РОССИЙСКОЙ СЛОВЕСНОСТИ': [],
    'СЕКРИТАРИАТ ПОПЕЧИТЕЛЬСКОГО СОВЕТА': [],
    'УПРАВЛЕНИЕ СЕКРИТАРИАТА И ПРОТОКОЛА': [],
    'ФЕДЕРАЛЬНЫЙ РЕСУРСНЫЙ ЦЕНТР ПСИХОЛОГИЧЕСКОЙ СЛУЖБЫ В СИСТЕМЕ ВЫСШЕГО ОБРАЗОВАНИЯ': [],
    'ЦЕНТР РАЗВИТИЯ ОБРАЗОВАНИЯ': [
        'ЛАБОРАТОРИЯ НАУЧНЫХ ПОДХОДОВ В УПРАВЛЕНИИ ОБРАЗОВАТЕЛЬНЫМИ СИСТЕМАМИ',
        'ЛАБОРАТОРИЯ РАЗВИТИЯ СОДЕРЖАНИЯ ОБРАЗОВАНИЯ',
        'ЛАБОРАТОРИЯ СОВРЕМЕННЫХ ФОРМ И МЕТОДОВ ОБУЧЕНИЯ',
        'ЛАБОРАТОРИЯ АНТРОПОЛОГИИ И ПЕДАГОГИЧЕСКОЙ КОМПАРАТИВИСТИКИ',
        'ЛАБОРАТОРИЯ СОЦИОЛОГИЧЕСКИХ ИССЛЕДОВАНИЙ В ОБРАЗОВАНИИ',
        'ЛАБОРАТИРИЯ РАЗВИТИЯ ЦИФРОВОЙ ОБРАЗОВАТЕЛЬНОЙ СРЕДЫ'],
    'ЦЕНТР РАЗВИТИЯ ВЫСШЕГО И СРЕДНЕГО ПРОФЕССИОНАЛЬНОГО ОБРАЗОВАНИЯ': [
        'ЛАБОРАТОРИЯ РАЗВИТИЯ СРЕДНЕГО ПРОФЕССИОНАЛЬНОГО ОБРАЗОВАНИЯ',
        'ЛАБОРАТОРИЯ РАЗВИТИЯ ВЫСШЕГО ПРОФЕССИОНАЛЬНОГО ОБРАЗОВАНИЯ',
        'ЛАБОРАТОРИЯ КООРДИНАЦИИ И МОНИТОРИНГА ДИССЕРТАЦИОННЫХ ИССЛЕДОВАНИЙ ПО ПЕДАГОГИЧЕСКИМ И ПСИХОЛОГИЧЕСКИМ НАУКАМ'
    ],
    'ЦЕНТР ОБРАЗОВАТЕЛЬНОГО ЗАКОНОДАТЕЛЬСТВА И ПРОБЛЕМ ПРАВОПРИМЕНЕНИЯ': [],
    'УЧЕБНЫЙ ЦЕНТР': [],
    'ЦЕНТР ДУХОВНОГО-НРАВСТВЕННОГО ВОСПИТАНИЯ И ТЕОЛОГИИ': [],
    'ЦЕНТР РАЗВИТИЯ ПЕДАГОГИЧЕСКОГО ОБРАЗОВАНИЯ': [
        'Лаборатория продвижения и пиар сопровождения педагогического образования'
    ],
    'ЦЕНТР РАЗВИТИЯ ИНКЛЮЗИВНОГО ОБРАЗОВАНИЯ': [
        'Лаборатория обучающихся с особыми образовательными потребностями',
        'Лаборатория содержания и форм подготовки специалистов с детьми с ОВЗ и инвалидностью'
    ],
    'ЦЕНТР ВОСПИТАНИЯ И РАЗВИТИЯ ЛИЧНОСТИ': [
        'ЛАБОРАТОРИЯ РАЗВИТИЯ ПРОСВЕТИТЕЛЬНОЙ ДЕЯТЕЛЬНОСТИ',
        'ЛАБОРАТОРИЯ ГУМАНИТАРНОГО ОБРАЗОВАНИЯ И ФОРМИРОВАНИЯ МИРОВОЗРЕНИЯ',
        'ЛАБОРАТОРИЯ СОВРЕМЕННЫХ ФОРМ И МЕТОДОВ ВОСПИТАНИЯ',
        'ЛАБОРАТОРИЯ СОВРЕМЕННЫХ ФОРМ И МЕТОДОВ ПРОФЕССИОНАЛЬНОГО САМООПРЕДЕЛЕНИЯ И ПРОФОРИЕНТАЦИИ',
        'Лаборатория научных основ изучения личности развития ребенка',
        'Лаборатория развития содержания воспитания и социализации',
        'Лаборатория психологического сопровождения развития личности в образовательном процессе'
    ],
    'ЦЕНТР РОДНЫХ ЯЗЫКОВ И КУЛЬТУР НАРОДОВ РФ': [],
    'ЦЕНТР РУССКОГО ЯЗЫКА И СЛАВИСТИКИ': [],
    'ЦЕНТР ЭКСПЕРТИЗЫ В ОБРАЗОВАНИИ': [],
    'ИНФОРМАЦИОННЫЙ ЦЕНТР "БИБЛИОТЕКА им. К.Д. УШИНСКОГО"': [
        'МЕТОДИЧЕСКИЙ ОТДЕЛ'
    ],
    'УПРАВЛЕНИЕ КООРДИНАЦИИ НАУЧНЫХ ИССЛЕДОВАНИЙ И ПОДГОТОВКИ КАДРОВ ВЫСШЕЙ КВАЛИФИКАЦИИ': [
        'ОТДЕЛ КООРДИНАЦИИ НАУЧНЫХ ИССЛЕДОВАНИЙ',
        'ОТДЕЛ КООРДИНАЦИИ ДЕЯТЕЛЬНОСТИ ПО ПОДГОТОВКЕ КАДРОВ ВЫСШЕЙ КВАЛИФИКАЦИИ',
        'ОТДЕЛ ПО РАБОТЕ С ПРОЕКТАМИ',
        'Отдел редакционно - издательской работы'
    ],
    'УПРАВЛЕНИЕ МЕЖДУНАРОДНОГО СОТРУДНИЧЕСТВА, СВЯЗЕЙ С ОБЩЕСТВЕННОСТЬЮ И СМИ': [
        'ОТДЕЛ МЕЖДУНАРОГО СОТРУДНИЧЕСТВА',
        'ОТДЕЛ ПО СВЯЗЯМ С ОБЩЕСТВЕННОСТЬЮ И СМИ'
    ],
    'ДЕПАРТАМЕНТ УПРАВЛЕНИЯ ДЕЛАМИ РАО': [
        'ОТДЕЛ КАДРОВ И ОХРАНЫ ТРУДА'
    ],
    'УПРАВЛЕНИЕ ПРАВОВОГО ОБЕСПЕЧЕНИЯ И ДЕЛОПРОИЗВОДСТВА': [
        'ОТДЕЛ ПРАВОВОГО ОБЕСПЕЧЕНИЯ И ИМУЩЕСТВЕННЫХ ОТНОШЕНИЙ',
        'ОТДЕЛ ДЕЛОПРОИЗВОДСТВА',
        'СЕКТОР НАУЧНОГО АРХИВА'
    ],
    'УПРАВЛЕНИЕ ФИНАНСОВОГО ПЛАНИРОВАНИЯ И БУХГАЛТЕРСКОГО УЧЕТА': [
        'ПЛАНОВЫЙ ОТДЕЛ',
        'ОТДЕЛ БУХГАЛТЕРСКОГО УЧЕТА И ОТЧЕТНОСТИ',
        'КОНТРАКТНАЯ СЛУЖБА',
        'ХОЗЯЙСТВЕННЫЙ ОТДЕЛ'
    ]
}